In [1]:
import polars as pl

In [2]:
df = pl.read_csv("C:/Users/user/Downloads/course_file.csv", has_header=True)

In [3]:
custom_schema = {
    'date_time': pl.Utf8,
    'userid': pl.Utf8,
    'dlbytes': pl.Int64,
    'ulbytes': pl.Int64,
    'clientip': pl.Utf8,
    'serverip': pl.Utf8,
    'country': pl.Utf8,
    'txn_time': pl.Float64,
    'http_method': pl.Utf8,
    'user_agent': pl.Utf8,
    'platform': pl.Utf8
}

In [4]:
df = pl.read_csv("C:/Users/user/Downloads/course_file.csv", has_header=True, dtypes=custom_schema)

In [5]:
platforms = ['Android', 'Linux']

df.select(
    pl.col('platform'), pl.col('dlbytes'),
    pl.when((pl.col('platform').is_in(platforms)) & (pl.col('dlbytes') > 5000))
    .then(pl.lit(1))
    .otherwise(pl.lit(0))
    .alias('platform_when')
).head(5)

platform,dlbytes,platform_when
str,i64,i32
"""Android""",872807,1
"""Android""",50898,1
"""Linux""",964276,1
"""Android""",212529,1
"""Windows""",553626,0


In [6]:
df.select(
    pl.col('ulbytes'), pl.col('dlbytes'),
    pl.when((pl.col('ulbytes') > 500000) & (pl.col('dlbytes') > 500000))
    .then(pl.lit('large'))\
    .when((pl.col('ulbytes') > 250000) & (pl.col('dlbytes') > 250000))
    .then(pl.lit('medium'))\
    .otherwise(pl.lit('small'))\
    .alias('transaction_size')
).head(5)

ulbytes,dlbytes,transaction_size
i64,i64,str
741526,872807,"""large"""
529504,50898,"""small"""
952420,964276,"""large"""
811887,212529,"""small"""
901428,553626,"""large"""


In [7]:
def calculate_bytes(row):
    try:
        dlbytes = int(row[3])
        ulbytes = int(row[4])
        return dlbytes + ulbytes
    except:
        return 1

df.with_columns(df.apply(calculate_bytes, return_dtype=pl.Int64)).rename({'apply': 'total_bytes'}).head(5)

date_time,userid,domain,dlbytes,ulbytes,clientip,serverip,country,txn_time,http_method,user_agent,platform,total_bytes
str,str,str,i64,i64,str,str,str,f64,str,str,str,i64
"""2023-10-04 11:…","""7773153683656""","""ryan-wells.co.…",872807,741526,"""142.132.219.11…","""62.42.184.180""","""England""",2.33,"""HTTPS""","""Mozilla/5.0 (W…","""Android""",1614333
"""2023-10-04 12:…","""1886351675683""","""hopkins.org""",50898,529504,"""184.205.48.78""","""152.123.41.39""","""Wales""",1.2,"""HTTP""","""Mozilla/5.0 (W…","""Android""",580402
"""2023-10-02 23:…","""1597721345356""","""evans.com""",964276,952420,"""189.30.60.163""","""68.171.236.18""","""Scotland""",1.32,"""HTTP""","""Mozilla/5.0 (W…","""Linux""",1916696
"""2023-10-03 09:…","""9766845800247""","""kelly.com""",212529,811887,"""13.144.79.35""","""63.141.80.109""","""England""",2.58,"""HTTPS""","""Opera/8.54.(X1…","""Android""",1024416
"""2023-10-01 08:…","""1546762097287""","""richardson-wal…",553626,901428,"""143.74.163.248…","""26.253.17.223""","""Wales""",1.53,"""HTTP""","""Mozilla/5.0 (M…","""Windows""",1455054


In [8]:
def makemilliseconds(row):
    try:
        txn_time = row[8]
        milliseconds = txn_time * 1000
        return milliseconds
    except:
        return 1
    
df.with_columns(df.apply(makemilliseconds, return_dtype=pl.Float64)).rename({'apply': 'txnMilliseconds'}).head(5)

date_time,userid,domain,dlbytes,ulbytes,clientip,serverip,country,txn_time,http_method,user_agent,platform,txnMilliseconds
str,str,str,i64,i64,str,str,str,f64,str,str,str,f64
"""2023-10-04 11:…","""7773153683656""","""ryan-wells.co.…",872807,741526,"""142.132.219.11…","""62.42.184.180""","""England""",2.33,"""HTTPS""","""Mozilla/5.0 (W…","""Android""",2330.0
"""2023-10-04 12:…","""1886351675683""","""hopkins.org""",50898,529504,"""184.205.48.78""","""152.123.41.39""","""Wales""",1.2,"""HTTP""","""Mozilla/5.0 (W…","""Android""",1200.0
"""2023-10-02 23:…","""1597721345356""","""evans.com""",964276,952420,"""189.30.60.163""","""68.171.236.18""","""Scotland""",1.32,"""HTTP""","""Mozilla/5.0 (W…","""Linux""",1320.0
"""2023-10-03 09:…","""9766845800247""","""kelly.com""",212529,811887,"""13.144.79.35""","""63.141.80.109""","""England""",2.58,"""HTTPS""","""Opera/8.54.(X1…","""Android""",2580.0
"""2023-10-01 08:…","""1546762097287""","""richardson-wal…",553626,901428,"""143.74.163.248…","""26.253.17.223""","""Wales""",1.53,"""HTTP""","""Mozilla/5.0 (M…","""Windows""",1530.0


In [9]:
df.select(
    pl.sum('dlbytes').alias('Total_DL_Bytes'),
    pl.min('dlbytes').alias('Min_DL_Bytes'),
    pl.max('dlbytes').alias('Max_DL_Bytes'),
    pl.std('dlbytes').alias('STD_DL_Bytes'),
    pl.median('dlbytes').alias('Median_DL_Bytes'),
    pl.mean('dlbytes').alias('Mean_DL_Bytes'),
    pl.first('dlbytes').alias('First_DL_Bytes'),
)

Total_DL_Bytes,Min_DL_Bytes,Max_DL_Bytes,STD_DL_Bytes,Median_DL_Bytes,Mean_DL_Bytes,First_DL_Bytes
i64,i64,i64,f64,f64,f64,i64
49999358573,61,999994,288565.822961,499479.5,499993.58573,872807


In [10]:
q = (
    df.lazy()
    .groupby('country')
    .agg(
        pl.sum('dlbytes').alias('sum_dlbytes')
    )
)

q.collect()

country,sum_dlbytes
str,i64
"""Ireland""",12496130770
"""England""",12629527260
"""Wales""",12506206681
"""Scotland""",12367493862


In [11]:
q = (
    df.lazy()
    .groupby('platform')
    .agg(
        pl.count().alias('row_count'),
        pl.sum('dlbytes'),
        pl.col('userid'),
        pl.first('http_method')
    )
    .sort('row_count', descending=True)
)

q.collect()

platform,row_count,dlbytes,userid,http_method
str,u32,i64,list[str],str
"""Linux""",20103,10045847375,"[""1597721345356"", ""3400788003398"", … ""4668345200091""]","""HTTP"""
"""Mac""",20054,10008386436,"[""8949163845658"", ""7007039082075"", … ""1070501782984""]","""HTTP"""
"""Android""",20011,10027385345,"[""7773153683656"", ""1886351675683"", … ""8113107789552""]","""HTTPS"""
"""Windows""",20005,10015562721,"[""1546762097287"", ""8035878244556"", … ""4509087590322""]","""HTTP"""
"""iOS""",19827,9902176696,"[""7214908687567"", ""1093994305313"", … ""0931956674865""]","""HTTP"""


In [12]:
df.sort('date_time', descending=False).select(
    pl.col('date_time'),
    pl.col('dlbytes'),
    pl.col('dlbytes').rolling_max(
            window_size=2
    ).alias('max')
).head(10)

date_time,dlbytes,max
str,i64,i64
"""2023-10-01 00:…",60865,null
"""2023-10-01 00:…",196289,196289
"""2023-10-01 00:…",350605,350605
"""2023-10-01 00:…",210027,350605
"""2023-10-01 00:…",523144,523144
"""2023-10-01 00:…",26727,523144
"""2023-10-01 00:…",453615,453615
"""2023-10-01 00:…",613899,613899
"""2023-10-01 00:…",115888,613899


In [13]:
df.select(
    pl.col('country'),
    pl.col('userid'),
    pl.col('dlbytes'),
    pl.col('dlbytes').sum().over('country').alias('windowbytes')
).head(5)

country,userid,dlbytes,windowbytes
str,str,i64,i64
"""England""","""7773153683656""",872807,12629527260
"""Wales""","""1886351675683""",50898,12506206681
"""Scotland""","""1597721345356""",964276,12367493862
"""England""","""9766845800247""",212529,12629527260
"""Wales""","""1546762097287""",553626,12506206681


In [14]:
df.filter(pl.col('http_method') == 'HTTP').head(5)

date_time,userid,domain,dlbytes,ulbytes,clientip,serverip,country,txn_time,http_method,user_agent,platform
str,str,str,i64,i64,str,str,str,f64,str,str,str
"""2023-10-04 12:…","""1886351675683""","""hopkins.org""",50898,529504,"""184.205.48.78""","""152.123.41.39""","""Wales""",1.2,"""HTTP""","""Mozilla/5.0 (W…","""Android"""
"""2023-10-02 23:…","""1597721345356""","""evans.com""",964276,952420,"""189.30.60.163""","""68.171.236.18""","""Scotland""",1.32,"""HTTP""","""Mozilla/5.0 (W…","""Linux"""
"""2023-10-01 08:…","""1546762097287""","""richardson-wal…",553626,901428,"""143.74.163.248…","""26.253.17.223""","""Wales""",1.53,"""HTTP""","""Mozilla/5.0 (M…","""Windows"""
"""2023-10-01 13:…","""8949163845658""","""lowe.com""",43595,77597,"""148.15.214.37""","""4.33.9.229""","""England""",1.68,"""HTTP""","""Mozilla/5.0 (L…","""Mac"""
"""2023-10-03 11:…","""3400788003398""","""watts.com""",342378,715125,"""142.37.27.131""","""41.152.174.228…","""Scotland""",2.92,"""HTTP""","""Opera/8.53.(Wi…","""Linux"""


In [15]:
df.filter((pl.col('platform') == 'Mac') & (pl.col('country') == 'England')).head(5)

date_time,userid,domain,dlbytes,ulbytes,clientip,serverip,country,txn_time,http_method,user_agent,platform
str,str,str,i64,i64,str,str,str,f64,str,str,str
"""2023-10-01 13:…","""8949163845658""","""lowe.com""",43595,77597,"""148.15.214.37""","""4.33.9.229""","""England""",1.68,"""HTTP""","""Mozilla/5.0 (L…","""Mac"""
"""2023-10-03 11:…","""1106150674730""","""turner.com""",728522,2711,"""1.120.139.1""","""194.164.198.19…","""England""",1.34,"""HTTPS""","""Mozilla/5.0 (M…","""Mac"""
"""2023-10-03 16:…","""7195638533371""","""lowe-brooks.bi…",583741,916671,"""188.226.195.12…","""201.95.79.16""","""England""",0.17,"""HTTP""","""Mozilla/5.0 (i…","""Mac"""
"""2023-10-01 10:…","""8815394944307""","""robinson.com""",332908,345730,"""80.77.128.175""","""77.238.12.133""","""England""",1.53,"""HTTP""","""Mozilla/5.0 (i…","""Mac"""
"""2023-10-04 16:…","""8093407508235""","""webster-franci…",943117,84462,"""138.64.12.233""","""157.237.117.16…","""England""",0.13,"""HTTPS""","""Mozilla/5.0 (M…","""Mac"""


find the 5 most heavy users

In [16]:
users = df.select(pl.col('userid'), pl.col('dlbytes'),
                  pl.col('ulbytes'),
                  (
                      pl.col('dlbytes') + pl.col('ulbytes')
                  ).alias('total_bytes')
)

In [17]:
q = (
    users.lazy()
    .groupby('userid')
    .agg(
        pl.sum('total_bytes').alias('total_bytes')
    )
    .sort('total_bytes', descending=True)
)

q.collect().head(5)

userid,total_bytes
str,i64
"""4741844108555""",3822427
"""3998031863277""",3818268
"""0805218874414""",3805682
"""2927557590468""",3798446
"""3261642863045""",3752013


use a window function to find the total number of ulbytes for each country

sum ulBytes and then work out the percent that each transaction represents

In [18]:
df2 = df.select(
    pl.col('country'),
    pl.col('userid'),
    pl.col('ulbytes'),
    pl.col('ulbytes').sum().over('country').alias('countrybytes')
)

In [19]:
df2.select(
    pl.col('userid'), pl.col('ulbytes'),
    ((
        pl.col('ulbytes') / pl.col('countrybytes')
    ) * 100).alias('pct')
).head(5)

userid,ulbytes,pct
str,i64,f64
"""7773153683656""",741526,0.005908
"""1886351675683""",529504,0.004208
"""1597721345356""",952420,0.007672
"""9766845800247""",811887,0.006468
"""1546762097287""",901428,0.007163
